# Building a ReAct Agent from Scratch

**Learning Objectives:**
- Understand how LLM API calls work
- Implement the ReAct (Reasoning + Acting) framework
- Connect an agent to real tools (Grasshopper)

---

## Part 1: Setup

In [ ]:
# Core imports for our ReAct agent
import re      # Regular expressions - used to parse LLM output for actions
import json    # For parsing API responses and tool results
import os      # For accessing environment variables

# Load environment variables from .env file (keeps API keys secure)
from dotenv import load_dotenv
load_dotenv()

# LLM provider - Groq offers fast inference with a generous free tier
from groq import Groq

# Alternative provider - Google's Gemini models
try:
    from google import genai
    from google.genai import types
    GEMINI_AVAILABLE = True
except ImportError:
    GEMINI_AVAILABLE = False
    print("Note: google-genai not installed. Run 'pip install google-genai' to use Gemini.")

# HTTP client that supports streaming - needed for MCP's SSE protocol
import httpx

In [ ]:
# =============================================================================
# LLM Provider Configuration
# Toggle USE_GEMINI to switch between providers. Both have free tiers!
# - Groq: https://console.groq.com/keys (14,400 requests/day)
# - Gemini: https://aistudio.google.com/apikey (1M token context)
# =============================================================================

USE_GEMINI = False  # Set to True for Gemini 2.5 Flash, False for Groq (Llama 3.3)

if USE_GEMINI:
    if not GEMINI_AVAILABLE:
        raise ImportError("google-genai not installed! Run: pip install google-genai")
    
    API_KEY = os.environ.get("GOOGLE_API_KEY")
    if not API_KEY:
        raise ValueError("GOOGLE_API_KEY not found! Add it to your .env file.")
    
    client = genai.Client(api_key=API_KEY)
    MODEL_NAME = "gemini-2.5-flash"
    print(f"Using Gemini: {MODEL_NAME}")
    
    # Quick test to verify connection
    response = client.models.generate_content(
        model=MODEL_NAME,
        contents="Tell me a short haiku about computational design."
    )
    print(response.text)
else:
    API_KEY = os.environ.get("GROQ_API_KEY")
    if not API_KEY:
        raise ValueError("GROQ_API_KEY not found! Copy .env.example to .env and add your key.")
    
    client = Groq(api_key=API_KEY)
    MODEL_NAME = "llama-3.3-70b-versatile"
    print(f"Using Groq: {MODEL_NAME}")
    
    # Quick test to verify connection
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": "Tell me a short haiku about computational design."}],
        temperature=0.0
    )
    print(response.choices[0].message.content)

### Provider Abstraction

The function below wraps our LLM calls. It automatically handles both **Groq** and **Gemini** based on the `USE_GEMINI` toggle above.

In [ ]:
def call_llm(messages: list[dict], temperature: float = 0) -> str:
    """
    Call the LLM with a list of messages.
    
    This abstraction layer allows us to switch providers easily.
    The message format follows the OpenAI standard used by most providers:
        {"role": "user" | "assistant" | "system", "content": "..."}
    """
    if USE_GEMINI:
        gemini_contents = []
        system_instruction = None
        
        for msg in messages:
            role = msg["role"]
            content = msg["content"]
            
            if role == "system":
                system_instruction = content
            elif role == "user":
                gemini_contents.append(types.Content(role="user", parts=[types.Part.from_text(text=content)]))
            elif role == "assistant":
                gemini_contents.append(types.Content(role="model", parts=[types.Part.from_text(text=content)]))
        
        config = types.GenerateContentConfig(
            temperature=temperature,
            max_output_tokens=4096,
            system_instruction=system_instruction,
        )
        
        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=gemini_contents,
            config=config
        )
        return response.text
    else:
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=messages,
            temperature=temperature,
            max_tokens=4096
        )
        return response.choices[0].message.content

---

## Part 2: The Agent Class

An agent is simply:
1. A **system prompt** that defines its behavior
2. A **message history** that tracks the conversation
3. A **method to call the LLM** and append responses

The key insight: **LLMs are stateless**. The agent "remembers" the conversation only because we maintain the message history and send it with every request.

In [ ]:
class Agent:
    """
    A conversational agent that maintains message history.
    
    The agent's behavior is defined by its system prompt. Each call to the agent
    adds to the conversation history, allowing the LLM to maintain context.
    """

    def __init__(self, system_prompt: str = ""):
        """Initialize the agent with an optional system prompt."""
        self.system_prompt = system_prompt
        self.messages = []

        # System message goes first - it sets the context for everything after
        if system_prompt:
            self.messages.append({"role": "system", "content": system_prompt})

    def __call__(self, user_message: str) -> str:
        """
        Send a message to the agent and get a response.
        
        The __call__ method allows using the agent like a function: agent("Hello")
        """
        # Add user message with correct role
        self.messages.append({"role": "user", "content": user_message})

        # Call LLM with entire conversation history
        response = call_llm(self.messages)

        # Add assistant response to history
        self.messages.append({"role": "assistant", "content": response})

        return response

    def reset(self):
        """Clear conversation history while keeping the system prompt."""
        self.messages = []
        if self.system_prompt:
            self.messages.append({"role": "system", "content": self.system_prompt})

In [ ]:
# Test the Agent class - the second question tests memory
test_agent = Agent("You are a helpful assistant. Be concise.")
print(test_agent("What is 2 + 2?"))
print(test_agent("What did I just ask you?"))  # This tests memory!

---

## Part 3: Connecting to Real Tools (Grasshopper MCP)

Now we'll connect our agent to **real tools** that can control Grasshopper.

The MCP (Model Context Protocol) server exposes these tools:
- `List_Python_Scripts` - Find script components on the canvas
- `Get_Python_Script` - Read a script's code
- `Edit_Python_Script` - Write/modify code
- `Get_Python_Script_Errors` - Check for compilation errors

**Important:** Make sure Grasshopper is running with the MCP server active!

In [ ]:
# MCP (Model Context Protocol) Connection
# This allows our agent to call tools in Grasshopper via HTTP/SSE

MCP_URL = "http://127.0.0.1:8089/mcp"  # Default Grasshopper MCP endpoint

def call_mcp_tool(tool_name: str, arguments: dict = None) -> dict:
    """Call a tool on the Grasshopper MCP server."""
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "tools/call",
        "params": {
            "name": tool_name,
            "arguments": arguments or {}
        }
    }
    
    try:
        with httpx.Client(timeout=30) as http_client:
            with http_client.stream("POST", MCP_URL, json=payload) as response:
                for line in response.iter_lines():
                    if line.startswith("data: "):
                        data = json.loads(line[6:])
                        if "result" in data:
                            return data["result"]
                        return data
                    elif line.strip() and not line.startswith(":"):
                        try:
                            return json.loads(line)
                        except:
                            pass
        return {"error": "No response received"}
    except Exception as e:
        return {"error": str(e)}

In [ ]:
# Fetch and filter available tools from MCP server

def get_available_tools() -> list:
    """Fetch tool definitions from the MCP server."""
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "tools/list",
        "params": {}
    }
    
    try:
        with httpx.Client(timeout=30) as http_client:
            with http_client.stream("POST", MCP_URL, json=payload) as response:
                for line in response.iter_lines():
                    if line.startswith("data: "):
                        data = json.loads(line[6:])
                        if "result" in data and "tools" in data["result"]:
                            return data["result"]["tools"]
        return []
    except Exception as e:
        print(f"Error fetching tools: {e}")
        return []

# Get all tools and filter to the Python script tools we need
ALL_TOOLS = get_available_tools()

TUTORIAL_TOOLS = [t for t in ALL_TOOLS if t["name"] in [
    "List_Python_Scripts",
    "Get_Python_Script", 
    "Edit_Python_Script",
    "Get_Python_Script_Errors"
]]

print(f"Found {len(ALL_TOOLS)} total tools, using {len(TUTORIAL_TOOLS)} for this tutorial:")
for tool in TUTORIAL_TOOLS:
    print(f"  - {tool['name']}: {tool['description'][:60]}...")

In [ ]:
# Test the MCP connection
result = call_mcp_tool("List_Python_Scripts")
print("Python scripts on canvas:", result)

---

## Part 4: The ReAct Framework

**ReAct** = **Re**asoning + **Act**ing

The key insight: LLMs can follow a structured loop if we tell them to:

1. **Thought** - Reason about what to do next
2. **Action** - Call a tool with specific input
3. **PAUSE** - Stop and wait for the tool result
4. **Observation** - Receive the tool's output
5. **Repeat** until ready to give final **Answer**

The magic is in the **prompt** - structured instructions create structured behavior.

In [ ]:
def build_react_prompt(tools: list) -> str:
    """
    Build a ReAct prompt from MCP tool definitions.
    
    The prompt has several key sections:
    1. Role definition - who the agent is
    2. Domain rules - Grasshopper-specific knowledge
    3. Tool descriptions - dynamically generated from MCP
    4. Workflow - the ReAct loop pattern
    5. Example - shows exactly what success looks like
    """
    
    tool_docs = []
    for tool in tools:
        name = tool["name"]
        desc = tool["description"].split('.')[0]
        params = tool.get("inputSchema", {}).get("properties", {})
        param_names = list(params.keys())
        tool_docs.append(f"- {name}: {desc}. Parameters: {param_names if param_names else 'none'}")
    
    tools_section = "\n".join(tool_docs)
    
    return f"""You are a Grasshopper Python scripting assistant that creates geometry.

<context>
You write Python scripts for Grasshopper (Rhino's visual programming tool).
Scripts run inside a Python component with predefined output variables.
</context>

<grasshopper_rules>
- Output geometry by assigning to variable 'a': a = my_points
- Import Rhino.Geometry as rg for geometry types
- Use rg.Point3d(x,y,z) for points, rg.Line() for lines, etc.
- Lists of geometry display as multiple objects
- Always assign final result to 'a' or it won't display!
</grasshopper_rules>

<tools>
{tools_section}
</tools>

<workflow>
1. Thought: reason about what to do
2. Action: call a tool, then PAUSE
3. Wait for Observation
4. After editing code, always check for errors
5. Only give Answer when code works
</workflow>

<format>
Action: ToolName {{"param": "value"}}
</format>

<example>
User: Create a spiral of 10 points

Thought: Find the script component first.
Action: List_Python_Scripts
PAUSE

Observation: [{{"id": "abc-123", "name": "Task"}}]

Thought: Write code for a spiral. Must assign to 'a' for output.
Action: Edit_Python_Script {{"componentId": "abc-123", "code": "import Rhino.Geometry as rg\\nimport math\\n\\npoints = []\\nfor i in range(10):\\n    angle = i * 0.5\\n    x = math.cos(angle) * i\\n    y = math.sin(angle) * i\\n    points.append(rg.Point3d(x, y, 0))\\n\\na = points"}}
PAUSE

Observation: Script updated

Thought: Check for errors.
Action: Get_Python_Script_Errors {{"componentId": "abc-123"}}
PAUSE

Observation: No errors

Answer: Created spiral of 10 points, assigned to output 'a'.
</example>
"""

REACT_PROMPT = build_react_prompt(TUTORIAL_TOOLS)
print(f"Prompt built ({len(REACT_PROMPT)} chars)")
print(REACT_PROMPT)

---

## Part 5: The ReAct Execution Loop

Now we need code that:
1. Parses the LLM's output to find Actions
2. Executes the corresponding tool
3. Feeds the result back as an Observation
4. Repeats until we get an Answer

In [ ]:
# Action Parser - extracts tool calls from LLM output using regex
# Format: "Action: ToolName {"param": "value"}"

ACTION_PATTERN = re.compile(r'^Action:\s*(\w+)\s*(\{.*\})?$', re.MULTILINE)

def parse_action(response: str) -> tuple[str, dict] | None:
    """
    Parse an action from the LLM response.
    Returns: (tool_name, arguments_dict) if action found, else None
    """
    match = ACTION_PATTERN.search(response)
    if match:
        tool_name = match.group(1)
        args_str = match.group(2)
        
        args = {}
        if args_str:
            try:
                args = json.loads(args_str)
            except json.JSONDecodeError:
                print(f"Warning: Could not parse arguments: {args_str}")
        
        return (tool_name, args)
    return None

In [ ]:
def execute_action(tool_name: str, args: dict) -> str:
    """
    Execute a tool and return the result as a string.
    Results are converted to strings because the LLM needs text.
    """
    result = call_mcp_tool(tool_name, args)
    
    if isinstance(result, dict):
        if "error" in result:
            return f"Error: {result['error']}"
        return json.dumps(result, indent=2)
    elif isinstance(result, list):
        return json.dumps(result, indent=2)
    else:
        return str(result)

In [ ]:
def query(question: str, max_turns: int = 10, verbose: bool = True) -> str:
    """
    Run the ReAct loop to answer a question.
    
    This orchestrates: Thought -> Action -> Observation -> repeat until Answer
    
    Key insight: We're having a conversation where "user" messages are
    actually observations from the real world (tool results)!
    """
    agent = Agent(REACT_PROMPT)
    next_prompt = question

    for turn in range(max_turns):
        if verbose:
            print(f"\n{'='*50}")
            print(f"Turn {turn + 1}")
            print(f"{'='*50}")

        response = agent(next_prompt)
        if verbose:
            print(response)

        # Check for final answer
        if "Answer:" in response and "PAUSE" not in response:
            answer_start = response.find("Answer:")
            return response[answer_start + 7:].strip()

        # Parse and execute action
        action_result = parse_action(response)
        if action_result:
            tool_name, args = action_result
            if verbose:
                print(f"\n>> Executing: {tool_name}({args})")

            observation = execute_action(tool_name, args)
            if verbose:
                print(f">> Result: {observation[:300]}...")

            next_prompt = f"Observation: {observation}"
        else:
            if verbose:
                print("\n>> No action found, prompting to continue...")
            next_prompt = "Continue with an Action or provide your Answer."

    return "Max turns reached."

---

## Part 6: Let's Run It!

**Make sure:**
1. Grasshopper is open
2. You have a Python 3 Script component on the canvas
3. The MCP server is running

In [ ]:
# Try different prompts!
user_prompt = """
Create a spiral staircase including steps
"""

result = query(user_prompt)

---

## Summary

### What You Built

1. **An Agent class** - maintains conversation history, calls the LLM
2. **A ReAct prompt** - tells the LLM to think, act, observe, repeat
3. **An execution loop** - parses actions, runs tools, feeds back results

### Key Takeaways

- **LLMs are stateless** - we maintain state with message history
- **Prompts are programs** - structured instructions create structured behavior  
- **Tools extend capabilities** - connect LLMs to real systems via MCP

### Next Steps

- Read the original ReAct paper: [Yao et al., 2022](https://arxiv.org/abs/2210.03629)
- [DeepLearning.AI Agentic AI Course](https://www.deeplearning.ai/courses/agentic-ai/)

---

## Bonus: Pretty Visualization with Rich

In [ ]:
from rich.console import Console
from rich.panel import Panel
from rich.syntax import Syntax

console = Console()

def extract_thought(response: str) -> str:
    """Extract the Thought section from LLM response."""
    lines = response.split('\n')
    thought_lines = []
    in_thought = False
    
    for line in lines:
        if line.strip().startswith('Thought:'):
            in_thought = True
            thought_lines.append(line.replace('Thought:', '').strip())
        elif in_thought and (line.strip().startswith('Action:') or line.strip() == 'PAUSE'):
            break
        elif in_thought:
            thought_lines.append(line.strip())
    
    return ' '.join(thought_lines).strip()

def query_pretty(question: str, max_turns: int = 10) -> str:
    """Run the ReAct loop with Rich visualization."""
    agent = Agent(REACT_PROMPT)
    next_prompt = question
    
    console.print(Panel(question, title="[bold white]User Request[/bold white]", border_style="white"))

    for turn in range(max_turns):
        console.print(f"\n[bold cyan]--- Turn {turn + 1} ---[/bold cyan]")

        response = agent(next_prompt)
        
        thought = extract_thought(response)
        if thought:
            console.print(Panel(thought, title="[bold]Thought[/bold]", border_style="blue"))

        if "Answer:" in response and "PAUSE" not in response:
            answer_start = response.find("Answer:")
            final_answer = response[answer_start + 7:].strip()
            console.print(Panel(final_answer, title="[bold]Final Answer[/bold]", border_style="green"))
            return final_answer

        action_result = parse_action(response)
        if action_result:
            tool_name, args = action_result
            args_str = json.dumps(args, indent=2) if args else "{}"
            console.print(Panel(
                f"[bold yellow]{tool_name}[/bold yellow]\n[dim]{args_str}[/dim]",
                title="[bold]Action[/bold]",
                border_style="green"
            ))

            observation = execute_action(tool_name, args)
            display_obs = observation[:500] + "..." if len(observation) > 500 else observation
            console.print(Panel(
                Syntax(display_obs, "json", theme="monokai", line_numbers=False),
                title="[bold]Observation[/bold]",
                border_style="yellow"
            ))

            next_prompt = f"Observation: {observation}"
        else:
            next_prompt = "Continue with an Action or provide your Answer."

    return "Max turns reached."

In [ ]:
# Try the pretty version!
query_pretty("Create a grid of 5x5 points")